In [32]:
import json,os
import numpy as np
import pandas as pd
from scipy.stats import kendalltau
import plotter_lib as pl
import networkx as nx

from datetime import datetime
import pytz

ligas = ["England","France","Germany", "Italy", "Spain"]
labels=["EN","FR","GE","IT","SP"]
ligaColor = {"England":"#9e0142", "France":"#f46d43", "Germany":"#66c2a5", "Spain":"#5e4fa2", "Italy":"#3288bd"}

formato = "%Y-%m-%d %H:%M:%S"

In [33]:
arx = '/home/chacoma/Lineas/Futbol/Ranking/ranking-futbol/data/league2team2rank.json'

l2t2r = json.load( open( arx,"r" ) )

genero un dict por liga con todos los partidos jugados, ordenados por fecha: col1:fecha, col2: ID de partido, col3:equipo1, col4:eq2, col4:winner (if 0 empate). Notar que cada linea en un partido

In [34]:
D={}

for liga in ligas:

    print (liga)

    arx = f'/home/chacoma/Lineas/Futbol/raw_data/pappalardo/matches/matches_{liga}.json'

    matches = json.load(open(arx,'r'))

    X=[]

    for i in range(len(matches)):


        match = matches[i]
        winner = str(matches[i]['winner'])

        fecha_str = match["dateutc"]
        naive_dt=datetime.strptime(fecha_str, formato)
        aware_dt_utc = naive_dt.replace(tzinfo=pytz.utc)
        timestamp = int(aware_dt_utc.timestamp())

        t1,t2 = list(match["teamsData"].keys())

        matchID = match['wyId']

        if winner!='0':
            X.append( [timestamp, matchID, l2t2r[liga][t1], l2t2r[liga][t2], l2t2r[liga][winner]] )
        else:
            X.append( [timestamp, matchID, l2t2r[liga][t1], l2t2r[liga][t2], 0] )

    #X = sorted(X, key= lambda a:a[0])

    D[liga] = pd.DataFrame(X, columns=["t","matchId","eq1","eq2","winner"]).sort_values(by="t").reset_index(drop=True)


England
France
Germany
Italy
Spain


por fecha se juegan 10 partido (9 en alemania), tomo todos los partidos hasta una fecha dada

In [35]:
Fecha=3

Dfs=[]

for liga in ligas:

    nlim = 10*Fecha if liga!='Germany' else 9*Fecha

    Dfs.append( D[liga][0:nlim] )


Partidos = pd.concat(Dfs, ignore_index=True)
    

In [36]:
Partidos

,t,matchId,eq1,eq2,winner
0,1502477100,2499719,6,9,6
1,1502537400,2499727,14,4,0
2,1502546400,2499726,19,17,0
3,1502546400,2499721,7,5,7
4,1502546400,2499728,12,20,20
...,...,...,...,...,...
142,1505037600,2565569,11,18,11
143,1505052900,2565574,16,10,16
144,1505061000,2565573,13,14,13
145,1505069100,2565568,6,5,5


Cargo los datos que tienen mas metricas por partido

In [37]:
arx_metrics = '../metrics.csv'

df = pd.read_csv(arx_metrics)

df.head()



,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
0,2499719,1609,1.046492,0.137967,0.008011,1.641360,0.404629,2.365172,0.475739,0.553544,-0.999813,2.012367,England
1,2499719,1631,-1.615181,-1.387848,-1.442053,0.599849,-1.344791,-0.581150,-0.981960,-1.274967,-0.617518,-2.056307,England
2,2499720,1625,1.715191,0.173965,-1.123182,2.261790,2.208721,2.062710,0.992867,1.951836,1.600498,0.787574,England
3,2499720,1651,-1.917373,-1.123628,-1.268779,-0.989425,-1.728302,-1.258772,-1.499088,-1.614897,-0.910111,-0.831514,England
4,2499721,1610,0.878730,0.072618,0.960670,0.500512,0.149394,0.352275,0.475739,-0.459076,-0.177330,0.393279,England


In [38]:

ids_a_buscar = Partidos["matchId"]

filtro = df['match'].isin(ids_a_buscar)

df_met_3Fecha = df[filtro].reset_index(drop=True)

print (df_met_3Fecha.shape)

df_met_3Fecha.head()



(294, 13)


,match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
0,2499719,1609,1.046492,0.137967,0.008011,1.641360,0.404629,2.365172,0.475739,0.553544,-0.999813,2.012367,England
1,2499719,1631,-1.615181,-1.387848,-1.442053,0.599849,-1.344791,-0.581150,-0.981960,-1.274967,-0.617518,-2.056307,England
2,2499720,1625,1.715191,0.173965,-1.123182,2.261790,2.208721,2.062710,0.992867,1.951836,1.600498,0.787574,England
3,2499720,1651,-1.917373,-1.123628,-1.268779,-0.989425,-1.728302,-1.258772,-1.499088,-1.614897,-0.910111,-0.831514,England
4,2499721,1610,0.878730,0.072618,0.960670,0.500512,0.149394,0.352275,0.475739,-0.459076,-0.177330,0.393279,England


In [39]:
df_met_3Fecha.to_csv( f"metricas_hasta_Fecha{Fecha}.csv", index=False )